In [6]:
import yfinance as yf
import pandas as pd
import mysql.connector
from datetime import datetime


sectors={
    'AAPL':'Technology','MSFT':'Technology','NVDA':'Technology',
    'JPM':'Finance','GS':'Finance','MS':'Finance',
    'JNJ':'Healthcare','PFE':'Healthcare','ABT':'Healthcare',
    'XOM':'Energy','CVX':'Energy','PSX':'Energy'
}

db=mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database='MultiSectorMarket'
)
cursor=db.cursor()


def calculate_technical_metrics(ticker):
    df = yf.download(ticker, period="60d", interval="1d", progress=False)
    
    if df.empty:
        return None

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    # Calculate Metrics
    df['Return'] = df['Close'].pct_change()
    df['MA50'] = df['Close'].rolling(window=50).mean()
    df['Vol'] = (df['High'] - df['Low']) / df['Close']
    
    # RSI Calculation
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    latest = df.iloc[-1]
    
    return {
        'price': float(latest['Close']),
        'volume': int(latest['Volume']),
        'return': float(latest['Return']) if pd.notnull(latest['Return']) else 0.0,
        'ma50': float(latest['MA50']) if pd.notnull(latest['MA50']) else 0.0,
        'volatility': float(latest['Vol']) if pd.notnull(latest['Vol']) else 0.0,
        'rsi': float(latest['RSI']) if pd.notnull(latest['RSI']) else 50.0
    }
# 3. Main Loop
cursor.execute("TRUNCATE TABLE Fact_StockData")
db.commit()
print("Old data cleared. Starting fresh refresh...")
for ticker, sector_name in sectors.items():
    print(f"Processing {ticker}...")
    
    cursor.execute("INSERT IGNORE INTO Dim_Companies (ticker, company_name, sector) VALUES (%s, %s, %s)", 
                   (ticker, ticker, sector_name))

    # Technical Metrics (RSI, MA50, Volatility)
    metrics = calculate_technical_metrics(ticker)
    
    if metrics:
        sql = """INSERT INTO Fact_StockData 
                 (ticker, date, close_price, volume, daily_return_pct, moving_avg_50, volatility_index, rsi_14) 
                 VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""
        
        values = (
            ticker, 
            datetime.now().strftime('%Y-%m-%d'), 
            metrics['price'], 
            metrics['volume'], 
            metrics['return'], 
            metrics['ma50'], 
            metrics['volatility'], 
            metrics['rsi']
        )
        
        try:
            cursor.execute(sql, values)
        except mysql.connector.Error as err:
            print(f"Error: {err}")

db.commit()
print("Succesfully Loaded Data")
cursor.close()
db.close()

Old data cleared. Starting fresh refresh...
Processing AAPL...
Processing MSFT...
Processing NVDA...
Processing JPM...
Processing GS...
Processing MS...
Processing JNJ...
Processing PFE...
Processing ABT...
Processing XOM...
Processing CVX...
Processing PSX...
Succesfully Loaded Data
